# Base example: How to Langchain
This is a simple example of how to use Langchain to split python files.

In [1]:
from langchain_community.document_loaders import DirectoryLoader, PythonLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
# from langchain_community.llms import CTransformers
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import NLTKTextSplitter
import torch

In [ ]:
loader = DirectoryLoader("../capstone/S35-Capstone-Backend",
                         glob="*.py", loader_cls=PythonLoader, recursive=True)
# interpret information in the documents
documents = loader.load()

for i, v in enumerate(documents):
    print(i, v)
# splitter = RecursiveCharacterTextSplitter(chunk_size=256,
#                                           chunk_overlap=20)
# texts = splitter.split_documents(documents)

# text_splitter = NLTKTextSplitter()
# texts = text_splitter.split_documents(documents)


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': "cuda"})

# create and save the local database
db = FAISS.from_documents(documents, embeddings)

template = """Use the following context to answer the user's question. 
Context: {context}
Question: {input}
"""

# load the language model
llm = OllamaLLM(model="llama3.1:8b",
                num_predict=-1,
                temperature=0.1)
# llm = CTransformers(model='./llama-2-7b-chat.ggmlv3.q8_0.bin',
#                     model_type='llama',
#                     gpu_layers=50,
#                     config={'max_new_tokens': 1024, 'temperature': 0.05})


# prepare a version of the llm pre-loaded with the local content
retriever = db.as_retriever(search_kwargs={'k': 5})
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'input'])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
qa_llm = create_retrieval_chain(retriever, combine_docs_chain)

# qa_llm = RetrievalQA.from_chain_type(llm=llm,
#                                      chain_type='stuff',
#                                      retriever=retriever,
#                                      return_source_documents=True,
#                                      chain_type_kwargs={'prompt': prompt})

# ask the AI chat about information in our local files

while True:
    inputP = input("What do you want to ask?\n")
    output = qa_llm.invoke({"input": inputP})
    print(output["answer"])


# Modified example: Custom Retriever

Testing if using a custom AST based retriever to split makes it better.

In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), '../ast_tokenizer/languages/python_ast')))
from python_ast import PythonASTDocumentLoader

In [7]:
loader = DirectoryLoader("../",
                         glob="*.py", loader_cls=PythonASTDocumentLoader, recursive=True)
# interpret information in the documents
documents = loader.load()

In [9]:
for i, v in enumerate(documents):
    print(i, v)

0 page_content='from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)' metadata={'relative_path': '../testing/test_splitter.py', 'start_offset': 0, 'end_offset': 90, 'block_type': 'others', 'block_name': 'Block at 0-90', 'block_args': [], 'parent_type': 'root', 'parent_name': '', 'functions_called': [], 'docstrings': [], 'comments': []}
1 page_content='from langchain_community.document_loaders import DirectoryLoader, PythonLoader' metadata={'relative_path': '../testing/test_splitter.py', 'start_offset': 91, 'end_offset': 169, 'block_type': 'others', 'block_name': 'Block at 91-169', 'block_args': [], 'parent_type': 'root', 'parent_name': '', 'functions_called': [], 'docstrings': [], 'comments': []}
2 page_content='from langchain_community.document_loaders.parsers.language.python import PythonSegmenter' metadata={'relative_path': '../testing/test_splitter.py', 'start_offset': 170, 'end_offset': 258, 'block_type': 'others', 'block_name': 'Block at 170-

# Hybrid Retrieval

Combine a sparse search with a dense search - BM25 + Embeddings
Reference Article: https://medium.com/etoai/hybrid-search-combining-bm25-and-semantic-search-for-better-results-with-lan-1358038fe7e6